In [1]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
from requests.auth import HTTPBasicAuth

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'

ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

# Actinia STAC Managment

Acinita is able to consume public and private Spatial Temporal Asset Catalogs (STAC)

## View STAC instances

Two Actinia routes seem to accomplish this task

### /stac

In [2]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   232  100   232    0     0  79779      0 --:--:-- --:--:-- --:--:--  113k
parse error: Invalid numeric literal at line 1, column 10


### /stac/instances

In [1]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/instances" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   398  100   398    0     0   1322      0 --:--:-- --:--:-- --:--:--  1322
{
  "defaultStac": {
    "path": "stac.defaultStac.rastercube.<stac_collection_id>"
  },
  "gee": {
    "path": "stac.gee.rastercube.<stac_collection_id>"
  },
  "naip": {
    "path": "stac.naip.rastercube.<stac_collection_id>"
  },
  "opentopography": {
    "path": "stac.opentopography.rastercube.<stac_collection_id>"
  },
  "planet": {
    "path": "stac.planet.rastercube.<stac_collection_id>"
  },
  "usgs": {
    "path": "stac.usgs.rastercube.<stac_collection_id>"
  }
}


## View STAC Catalogs

### /stac/catalogs/catalog.json

In [2]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/catalogs/catalog.json" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   232  100   232    0     0   113k      0 --:--:-- --:--:-- --:--:--  113k
parse error: Invalid numeric literal at line 1, column 10


## View STAC collections

STAC collection 

> Spec: [https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md](https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md)

### /stac/collections

In [8]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/collections"

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>


## View STAC collection

### /stac/collections/{stac_collection_id}

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/collections/stac.defaultStac.rastercube.landsat-8-l1-c1" | jq

# Demo

We will now add the Planet SkySat STAC from [https://stacindex.org/catalogs/planet-labs-stac-catalog#/](https://stacindex.org/catalogs/planet-labs-stac-catalog#/)

## 1) Add new STAC instance

In [2]:
!curl -u 'actinia-gdi:actinia-gdi' -X POST "http://localhost:8088/api/v3/stac/instances/" -H "Content-Type: application/json" \
   -d '{"stac_instance_id": "3dep"}' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   101  100    73  100    28    250     96 --:--:-- --:--:-- --:--:--   345
{
  "StacInstance": {},
  "message": "The Instance has been added successfully"
}


Now lets view the new instance by requesting it with the **/stac/instances/{stac_instance_id}** GET request.

In [3]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/instances/3dep" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     3  100     3    0     0     10      0 --:--:-- --:--:-- --:--:--    10
{}


The request should return an empty response object because we have not added any collections to the data yet.

## 2) Add a new collection to an instance

To add a new collect to the opentopography instace we need to make a POST request to the **/stac/collections** routes specifying the `stac_instance_id` and setting the `stac_url`.

In our case the `stac_instance_id = planet` and the `stac_url = https://www.planet.com/data/stac/open-skysat-data/collection.json`

The request body will look like 

```json
{
    "stac_instance_id": "opentopography", 
    "stac_url": "https://www.planet.com/data/stac/open-skysat-data/collection.json"}
}
```

In [4]:
!curl -u 'actinia-gdi:actinia-gdi' -X POST "http://localhost:8088/api/v3/stac/collections" -H "Content-Type: application/json" \
   -d '{"stac_instance_id": "3dep", "stac_url": "https://planetarycomputer.microsoft.com/api/stac/v1/collections/3dep-seamless"}' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   396  100   275  100   121    104     46  0:00:02  0:00:02 --:--:--   150
{
  "StacCollection": {
    "stac.3dep.rastercube.3dep-seamless": {
      "href": "api/v3/stac/collections/stac.3dep.rastercube.3dep-seamless",
      "root": "https://planetarycomputer.microsoft.com/api/stac/v1/collections/3dep-seamless"
    }
  },
  "message": "The STAC Collection has been added successfully"
}


## 3) View the new collection

Now lets view the new collection using the GET **/stac/collections/{stac_collection_id}** by setting the route `stac_collection_id` route parameter to `stac.planet.rastercube.planet-stac-skysat`.

In [5]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/collections/stac.3dep.rastercube.3dep-seamless" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3258  100  3258    0     0  11594      0 --:--:-- --:--:-- --:--:-- 11594
{
  "assets": {
    "thumbnail": {
      "href": "https://ai4edatasetspublicassets.blob.core.windows.net/assets/pc_thumbnails/threedep.png",
      "roles": [
        "thumbnail"
      ],
      "title": "USGS 3DEP",
      "type": "image/png"
    }
  },
  "description": "U.S.-wide digital elevation data at horizontal resolutions ranging from one to sixty meters.\\n\\nThe [USGS 3D Elevation Program (3DEP) Datasets](https://www.usgs.gov/core-science-systems/ngp/3dep) from the [National Map](https://www.usgs.gov/core-science-systems/national-geospatial-program/national-map) are the primary elevation data product produced and distributed by the USGS. The 3DEP program provides raster elevation data for the conterminous United States, Alaska, Hawaii, and the is

In [19]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/locations/nc_spm_08/mapsets/PERMANENT/strds" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1584  100  1584    0     0   1463      0  0:00:01  0:00:01 --:--:--  1463
{
  "accept_datetime": "2022-04-22 03:36:29.941543",
  "accept_timestamp": 1650598589.9415414,
  "api_info": {
    "endpoint": "syncstrdslisterresource",
    "method": "GET",
    "path": "/api/v3/locations/nc_spm_08/mapsets/PERMANENT/strds",
    "request_url": "http://localhost:8088/api/v3/locations/nc_spm_08/mapsets/PERMANENT/strds"
  },
  "datetime": "2022-04-22 03:36:30.641870",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "1": {
        "inputs": {
          "column": "name",
          "type": "strds",
          "where": "mapset='PERMANENT'"
        },
        "module": "t.list"
      }
    }
  ],
  "process_log": [
    {
      "executable": "t.list",
      "id": "1",
      "parameter": [

In [ ]:
# Collection
https://earthengine-stac.storage.googleapis.com/catalog/USGS/NLCD_RELEASES/USGS_NLCD_RELEASES_2019_REL_NLCD.json

In [1]:
!curl -u 'actinia-gdi:actinia-gdi' -X POST "http://localhost:8088/api/v3/stac/instances/" -H "Content-Type: application/json" \
   -d '{"stac_instance_id": "gee"}' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   100  100    73  100    27    219     81 --:--:-- --:--:-- --:--:--   301
{
  "StacInstance": {},
  "message": "The Instance has been added successfully"
}


In [3]:
!curl -u 'actinia-gdi:actinia-gdi' -X POST "http://localhost:8088/api/v3/stac/collections" -H "Content-Type: application/json" \
   -d '{"stac_instance_id": "gee", "stac_url": "https://earthengine-stac.storage.googleapis.com/catalog/USGS/USGS_NLCD.json"}' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   194  100    76  100   118     98    153 --:--:-- --:--:-- --:--:--   251
{
  "message": "Please check the URL provided (Should be a STAC Collection)."
}


## 4) Run a process chain using STAC data

In [47]:
!ace location="nc_spm_08" mapset="basin_test" script="./scripts/stac.sh" -d | jq

{
  "version": "1",
  "list": [
    {
      "module": "g.region",
      "id": "g.region_1804289383",
      "inputs": [
        {
          "param": "raster",
          "value": "elevation"
        },
        {
          "param": "res",
          "value": "10"
        }
      ]
    },
    {
      "module": "r.in.gdal",
      "id": "r.in.gdal_1804289383",
      "flags": "r",
      "inputs": [
        {
          "param": "input",
          "value": "/vsicurl/https://storage.googleapis.com/tomorrownow-actinia-dev/nlcd/nlcd_2019_cog.tif"
        },
        {
          "param": "memory",
          "value": "300"
        },
        {
          "param": "offset",
          "value": "0"
        },
        {
          "param": "num_digits",
          "value": "0"
        }
      ],
      "outputs": [
        {
          "param": "output",
          "value": "nlcd_2019_cog"
        }
      ]
    },
    {
      "module": "r.slope.aspect",
      "id": "r.slope.aspect_1804289383",
      "inputs": [

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X POST "http://localhost:8088/api/v3/stac/collections" -H "Content-Type: application/json" \
   -d '{"stac_instance_id": "3dep", "stac_url": "https://planetarycomputer.microsoft.com/api/stac/v1/collections/3dep-seamless"}' | jq

In [44]:
from datetime import datetime

import pystac
print(pystac.__version__)

from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.view import ViewExtension
# from pystac.extensions.query import QueryExtension
stac_name = "3dep-seamless"
# bbox = [['597919.0778758611', '237972.1108290927', '604459.5119219327', '243782.34364209961']]
# bbox = ['597919.0778758611', '237972.1108290927', '604459.5119219327', '243782.34364209961']
bbox = ['-79.12940583962109', '35.89513100938877', '-79.05699434319902', '35.947553915652655']
url = "http://localhost:8088/api/v3/stac/collections/stac.3dep.rastercube.3dep-seamless"
interval = [['2021-09-09', '2021-09-12']]
stac_from_actinia = requests.get(url, auth=ACTINIA_AUTH)
stac_json = stac_from_actinia.json()
# print(stac_json)
stac_root_search=None
for item in stac_json["links"]:
    if item["rel"] == "root":
        stac_url = item["href"]
        # print(stac_url)
        if "?" in stac_url:
            stac_url = stac_url.split("?")[0]
            # print(stac_url)
            
        if stac_url.endswith("/"):
            stac_root_search = stac_url + "search"
        else:
            stac_root_search = stac_url + "/search"
       
# print(stac_root_search)
search_body = {
    "collections": [stac_name],
}
# search_body["query"] = filter

search_body["bbox"] = bbox

search_body["interval"] = interval
# print(search_body)

stac_search = requests.post(
    stac_root_search,
    json=search_body
)

# print(stac_search.content)

full_filtered_result = stac_search.json()

if "features" in full_filtered_result:
    # full_filtered_result
    
    for feature in full_filtered_result['features']:
        for key, value in feature["assets"].items():
            if key == 'data':
                print('Key', key)
                print(value['title'])
                print(value['href'])
           
                    
        # for name_id, url in value.items():
        #     output_name = stac_name + "_" + key + "_" + name_id
        #     # From Here Onwards, the Process build starts
        #     exec_params = ["input=%s" % "/vsicurl/"+url,"output=%s" % output_name,"-o"]

1.4.0
Key data
USGS 1/3 arc-second n36w080 1 x 1 degree
https://ai4edataeuwest.blob.core.windows.net/3dep/Elevation/13/TIFF/n36w080/USGS_13_n36w080.tif
Key data
USGS 1 arc-second n36w080 1 x 1 degree
https://ai4edataeuwest.blob.core.windows.net/3dep/Elevation/1/TIFF/n36w080/USGS_1_n36w080.tif


In [45]:
!importer --help

Actinia importer module supporting raster and vector data.

Usage:
 importer [raster=name] [vector=name] [--help] [--verbose] [--quiet]
   [--ui]

Parameters:
  raster   Name of raster map to be imported by actinia
  vector   Name of vector map to be imported by actinia


In [46]:
!importer raster=USGS_13_n36w080@https://ai4edataeuwest.blob.core.windows.net/3dep/Elevation/13/TIFF/n36w080/USGS_13_n36w080.tif